In [ ]:
# step 1
!git clone https://github.com/jgabrielgv/fruit-quality-classifier.git

In [ ]:
# step 2
import tensorflow.keras

model = tensorflow.keras.models.load_model('fruit-quality-classifier/models/fruit_predictor/fruit_type_prediction_model.h5')

In [ ]:
# step 4
#image = cv2.imread('fruit-quality-classifier/samples/banano_maduro.png')
#image = cv2.imread('fruit-quality-classifier/samples/banano_verde.png')
#image = cv2.imread('fruit-quality-classifier/samples/naranja_madura.png')
image = cv2.imread('fruit-quality-classifier/samples/naranja_verde.png')

fruit_ripeness = resolve_fruit_ripeness(image)
print("Nivel de maduración:")
print(fruit_ripeness)

In [5]:
# step 3
from __future__ import division
import io
import os
import random
import cv2
import numpy as np
import time
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image, ImageOps

kernelOpen=np.ones((5,5))
kernelClose=np.ones((20,20))

# The name of the image file to annotate
i=time.strftime("%d-%m-%y_%H-%M-%S")

# capture image
# camera = cv2.VideoCapture(0)
#return_value, image = camera.read()
def predict_fruit_type(image):
    # Disable scientific notation for clarity
    np.set_printoptions(suppress=True)

    # Load the model
    # model = tensorflow.keras.models.load_model('./models/fruit_predictor/keras_model.h5')

    # Create the array of the right shape to feed into the keras model
    # The 'length' or number of images you can put into the array is
    # determined by the first position in the shape tuple, in this case 1.
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
    # image = Image.open('test_pimport tensorflow.kerashoto.jpg')

    #resize the image to a 224x224 with 01-08-20_09-32-56the same strategy as in TM2:
    #resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = cv2.resize(image, size, interpolation = cv2.INTER_AREA)
    #image = ImageOps.fit(image, size, Image.ANTIALIAS)
    
    #turn the image into a numpy array
    image_array = np.asarray(image)

    # display the resized image
    # image.show()

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # run the inference
    prediction = model.predict(data)
    return prediction

def resolve_fruit_ripeness(image):
  #cv2.imwrite("images/" + i + '.jpeg', image)
  #del(camera)

  fruit_prediction = predict_fruit_type(image)
  fruit_type = resolve_fruit_type(fruit_prediction)
  print("El tipo de fruta es:")
  print(fruit_type)

  return resolve_banana_ripeness(image)

def resolve_banana_ripeness(image):
  frame=image
  edge_img=deepcopy(image)

  # finds edges in the input image image and
  # marks them in the output map edges
  edged = cv2.Canny(edge_img,50,100)
  edged = cv2.dilate(edged, None, iterations=1)
  edged = cv2.erode(edged, None, iterations=1)

  # find contours in the edge map
  cnts, h = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

  max_contA=cv2.contourArea(cnts[0])
  max_cont=max(cnts,key=cv2.contourArea)

  for i in range(len(cnts)):
        x,y,w,h=cv2.boundingRect(max_cont)
        cv2.rectangle(edge_img,(x,y),(x+w,y+h),(0,0,255), 2)
  croppedk=frame[y:y+h,x:x+w]

  # Display the fruit 
  #cv2.imshow('Edges',edge_img)

  frame=edge_img

  # converting BGR to HSV
  hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
      
  # define range of red color in HSV
  lower_red = np.array([0,50,50]) #verde
  upper_red = np.array([10,255,255]) #celesteeeee
      
  # create a red HSV colour boundary and 
  # threshold HSV image
  redmask1 = cv2.inRange(hsv, lower_red, upper_red)
      
  # define range of red color in HSV
  lower_red = np.array([170,50,50]) #rojo #170,50,50
  upper_red = np.array([180,255,255]) #celeste claro
      
  # create a red HSV colour boundary and 
  # threshold HSV image
  redmask2 = cv2.inRange(hsv, lower_red, upper_red)
  
  redmask=redmask1+redmask2
  #maskOpen=cv2.morphologyEx(redmask,cv2.MORPH_OPEN,kernelOpen)
  #maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

  #maskFinal=maskClose
  #cv2.imshow('Red_Mask:',maskFinal)


  cnt_r=0
  for r in redmask:
    cnt_r=cnt_r+list(r).count(255)
  # print ("Redness ",cnt_r)

  lower_green=np.array([50,50,50])
  upper_green=np.array([70,255,255])
  greenmask = cv2.inRange(hsv, lower_green, upper_green)
  #cv2.imshow('Green_Mask:',greenmask)
  cnt_g=0
  for g in greenmask:
    cnt_g=cnt_g+list(g).count(255)
  #print ("Greenness ",cnt_g)

  lower_yellow=np.array([20,50,50])
  upper_yellow=np.array([30,255,255])
  yellowmask = cv2.inRange(hsv, lower_yellow, upper_yellow)
  #cv2.imshow('Yellow_Mask:',yellowmask)
  cnt_y=0
  for y in yellowmask:
    cnt_y=cnt_y+list(y).count(255)
  #print ("Yellowness ",cnt_y)


  #Calculate ripeness
  tot_area=cnt_r+cnt_y+cnt_g
  rperc=cnt_r/tot_area
  yperc=cnt_y/tot_area
  gperc=cnt_g/tot_area


  #Adjust the limits for your fruit
  glimit=0.35
  ylimit=0.65

  #print ("yellow percentage")
  #print (yperc)

  if yperc>ylimit:
    return "Muy madura"
  elif gperc<glimit:
    return "Poco madura"
  else:
    return "Medianamente madura"
  # Wait for any key to close
  #while True: 
  #  k = cv2.waitKey(5) & 0xFF
  #  if k==27:
  #    break
  # De-allocate any associated memory usage
  #cv2.waitKey(0)
  #cv2.destroyAllWindows()

def get_available_fruits():
  return [ "banana" "orange" ]

def resolve_fruit_type(fruit_prediction):
  # e.g.: [[33.4, 66.6]]
  print(fruit_prediction)
  prediction_array = fruit_prediction[0]

  # TODO: do this generic
  banana, orange = prediction_array

  # TODO: check when it's 50/50 (literally)
  return "banano" if banana > orange else "naranja"
